# How can we control the increasing number of accidents in New York?

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import base64

## Introduction

**Business Context.** The city of New York has seen a rise in the number of accidents on the roads in the city. They would like to know if the number of accidents have increased in the last few weeks. For all the reported accidents, they have collected details for each accident and have been maintaining records for the past year and a half (from January 2018 to August 2019). 

The city has contracted you to build visualizations that would help them identify patterns in accidents, which would help them take preventive actions to reduce the number of accidents in the future. They have certain parameters like borough, time of day, reason for accident, etc. Which they care about and which they would like to get specific information on.

**Business Problem.** Your task is to format the given data and provide visualizations that would answer the specific questions the client has, which are mentioned below.

**Analytical Context.** You are given a CSV file (stored in the already created ```data``` folder) containing details about each accident like date, time, location of the accident, reason for the accident, types of vehicles involved, injury and death count, etc. The delimiter in the given CSV file is `;` instead of the default `,`. You will be performing the following tasks on the data:

1. Extract additional borough data stored in a JSON file
2. Read, transform, and prepare data for visualization
3. Perform analytics and construct visualizations of the data to identify patterns in the dataset
        
The client has a specific set of questions they would like to get answers to. You will need to provide visualizations to accompany these:

1. How have the number of accidents fluctuated over the past year and a half? Have they increased over the time?
2. For any particular day, during which hours are accidents most likely to occur?
3. Are there more accidents on weekdays than weekends?
4. What are the accidents count-to-area ratio per borough? Which boroughs have disproportionately large numbers of accidents for their size?
5. For each borough, during which hours are accidents most likely to occur?
6. What are the top 5 causes of accidents in the city? 
7. What types of vehicles are most involved in accidents per borough?
8. What types of vehicles are most involved in deaths?

**Note:** To solve this extended case, please read the function docstrings **very carefully**. They contain information that you will need! Also, please don't include `print()` statements inside your functions (they will most likely produce an error in the test cells).

## Fetching the relevant data

The client has requested analysis of the accidents-to-area ratio for boroughs. Borough data is stored in a JSON file in the ```data``` folder (this file was created using data from [Wikipedia](https://en.wikipedia.org/wiki/Boroughs_of_New_York_City)).

### Question

Use the function ```json.load()``` to load the file ```borough_data.json``` as a dictionary.

**Answer.** One possible solution is given below:

In [ ]:
with open('data/borough_data.json') as f:
    borough_data=json.load(f)
borough_data

### Question

Similarly, use the `pandas` function ```read_csv()``` to load the file ```accidents.csv``` as a DataFrame. Name this DataFrame ```df```. 

**Answer.** One possible solution is given below:

In [ ]:
with open('data/accidents.csv') as f:
    df=pd.read_csv(f, delimiter=';')

## Overview of the data

Let's go through the columns present in the DataFrame:

In [ ]:
df.columns

We have the following columns:

1. **BOROUGH**: The borough in which the accident occurred
2. **COLLISION_ID**: A unique identifier for this collision
3. **CONTRIBUTING FACTOR VEHICLE (1, 2, 3, 4, 5)**: Reasons for the accident
4. **CROSS STREET NAME**: Nearest cross street to the location of the accident
5. **DATE**: Date of the accident
6. **TIME**: Time of the accident
7. **LATITUDE**: Latitude of the accident
8. **LONGITUDE**: Longitude of the accident
9. **NUMBER OF (CYCLISTS, MOTORISTS, PEDESTRIANS) INJURED**: Injuries by category
10. **NUMBER OF (CYCLISTS, MOTORISTS, PEDESTRIANS) KILLED**: Deaths by category
11. **ON STREET NAME**: Street where the accident occurred
13. **VEHICLE TYPE CODE (1, 2, 3, 4, 5)**: Types of vehicles involved in the accident
14. **ZIP CODE**: Zip code of the accident location

### Exercise 1

Since 2014, New York City has been implementing a road safety plan named [Vision Zero](https://www1.nyc.gov/content/visionzero/pages/). It aims to reduce the number of traffic deaths to *zero* by the end of 2024. The plan is creating new and enhancing current safety measures, some of these include:

<ul>
A. Automated pedestrian detection<br>
B. Road safety audits at high risk locations<br>
C. Expansion in the cycle track network<br>
D. Targeted education and awareness initiatives<br>
E. Creation of pedestrian refuge islands<br>
F. Launch Integrated Data-Driven Speed Reducer Program (speed humps & speed cushions)<br>
</ul>

Which of these initiatives could directly benefit from an analysis of the data provided?

**Note:** In this notebook, whenever you are asked to write text, use the cell below the question cell to write your answer there. If you write in the same cell as the question, your answer will not be recorded.

=== BEGIN MARK SCHEME ===

**Answer.** B, D, and F.

- Automated pedestrian detection: NO

The data provided contains the exact location, the number of pedestrians injured/killed, and the contributing factor of the vehicles in each accident. We could study the accidents where pedestrians were injured or killed. This *could* help to place cameras at important spots, where the automatic detection may be needed. However, for the *implementation* of automated pedestrian detection, a data set of images of crosswalks and intersections is more crucial.

- Road safety audits at high risk locations: YES

As discussed, our data can help with the recognition of problematic spots. We can check the street names where a high number of accidents have occurred in the past, then inspect the area to see what extra measures should be implemented. 

- Expansion in the cycle track network: NO

Our data contains the number of cyclists injured/killed. We could determine the spots where these are happening the most and then improve the safety measures around the area. However, we would need more data to determine if these locations have a high volume of cyclists. Thus the data may not be that beneficial for making decisions on expanding the cycle network.

- Targeted education and awareness initiatives: YES

The data provides the contributing factor of the accidents along with the type of vehicles that were involved in them. We can then study which types of vehicles cause the most number of accidents and organize educational initiatives around this.

- Creation of pedestrian refuge islands: NO

Pedestrian islands are usually created on streets that are very wide. Thus, the information provided might not be that beneficial for this initiative, as we do not have information on the width/size of streets.

- Launch Integrated Data-Driven Speed Reducer Program (speed humps & speed cushions): YES

One of the contributing factors in accidents provided in the data is speeding. We can check if these accidents occur in highly transited areas. Thus, our data might be useful for the implementation of this initiative.

=== END MARK SCHEME ===

**Your answer here**.

## Answering the client's questions

Let's go ahead and answer each of the client's questions.

### Exercise 2

#### 2.1

Group the available accident data by month.

**Hint**: You may find the `pandas` functions [```pd.to_datetime()```](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) and [```dt.to_period()```](https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.to_period.html) useful.

In [ ]:
def ex_2(df):
    """
    Group accidents by month
    
    Arguments:
    `df`: A pandas DataFrame
    
    Outputs:
    `monthly_accidents`: The grouped Series
    """
    
    ### BEGIN SOLUTION
    
    #First we need to change the format of the column DATE to datetime:
    df['DATE']=pd.to_datetime(df['DATE'])

    #Then, check whether the number of accidents has increased over time
    monthly_accidents = df.groupby(df['DATE'].dt.to_period('M')).size()
    
    ### END SOLUTION
    
    return monthly_accidents

#### 2.2
##### 2.2.1

Generate a line plot of accidents over time.

In [ ]:
ex_2(df).plot.line()

##### 2.2.2

Has the number of accidents increased over the past year and a half?

=== BEGIN MARK SCHEME ===

**Answer.**

You get full points if you included some interpretation like this:

> The line graph clearly shows that there is no obvious uptrend in accidents over time.

=== END MARK SCHEME ===

**Your answer here**.

### Exercise 3

From the plot above, which months seem to have the least number of accidents? What do you think are the reasons behind this?

=== BEGIN MARK SCHEME ===

**Answer.** You get full points if you noticed at least one of the following (otherwise, nothing):

* August of 2019 is the month with the lowest number of accidents (this is is probably due to the fact that there is not a complete record for this month, since the latest record in August is from the 24th, namely, the entire final week of data is missing).

* Disregarding this month, the lowest number of accidents appear to happen in February. This is the month of the year with the lowest temperatures and road conditions might get a little complicated. Thus, people may prefer to use public transportation and avoid going out as much as possible due to the cold weather. This is in contrast with summer, where people are prone to go out and more accidents may occur.

=== END MARK SCHEME ===

**Your answer here**.

### Exercise 4

#### 4.1

Create a new column `HOUR` based on the data from the `TIME` column.

**Hint:** You may find the ```dt.hour``` accessor useful.

In [ ]:
def ex_4(df):
    """
    Group accidents by hour of day
    
    Arguments:
    `df`: A pandas DataFrame
    
    Outputs:
    `hourly_accidents`: The grouped Series

    """
    
    ### BEGIN SOLUTION
    
    # First change the format of the TIME column
    df['TIME']=pd.to_datetime(df['TIME'])

    # Create a new hour column 
    df['HOUR'] = df['TIME'].dt.hour

    # Find out how the number of accidents varies across hours. 
    hourly_accidents = df.groupby('HOUR').size()
    
    ### END SOLUTION
    
    return hourly_accidents

#### 4.2
##### 4.2.1
 
Plot a bar graph of the distribution per hour throughout the day.

In [ ]:
ex_4(df).plot.bar()

##### 4.2.2
How does the number of accidents vary throughout a single day?

=== BEGIN MARK SCHEME ===

**Answer.**

You get full points if you included some interpretation like this:

> From this, we see that more accidents occur in the afternoon (2 - 6 PM) than at other times of day.

=== END MARK SCHEME ===


**Your answer here**.

### Exercise 5

In the above question we have aggregated the number accidents per hour disregarding the date and place of occurrence. What criticism would you give to this approach?


=== BEGIN MARK SCHEME ===

**Answer.** You get half points if you mentioned one of these and full points if you mentioned both:

* Aggregating the data will be a problem if the disregarded variables exhibit large degrees of variation and are not distributed very evenly. It may be the case that the concentration of accidents in some zones of the city happen at different times of the day, yet by aggregating the data we may falsely conclude that accidents are constant in the entire city.
* Probably, residential zones are busy around 5 - 7pm, but commercial or manufaturing zones might be busy around 3 - 5pm. This information is important if the client is interested in designing a deployment plan for police officers to litigate traffic accidents, yet will be missed if we aggregate across all places of occurrence.

=== END MARK SCHEME ===

**Your answer here**.

### Exercise 6


#### 6.1

Calculate the number of accidents by day of the week.

**Hint:** You may find the ```dt.weekday``` accessor useful.

In [ ]:
def ex_6(df):
    """
    Group accidents by day of the week
    
    Arguments:
    `df`: A pandas DataFrame
    
    Outputs:
    `weekday_accidents`: The grouped Series
    """
    
    ### BEGIN SOLUTION
    
    # First change the format of the DATE column
    df['DATE']=pd.to_datetime(df['DATE'])
    
    # Extract the weekday
    df['WEEKDAY'] = df['DATE'].dt.weekday
    
    # Grouping
    weekday_accidents = df.groupby('WEEKDAY').size()
        
    ### END SOLUTION
    
    return weekday_accidents

#### 6.2
##### 6.2.1
Plot a bar graph based on the accidents count by day of the week.

In [ ]:
ex_6(df).plot.bar()

##### 6.2.2

How does the number of accidents vary throughout a single week?

=== BEGIN MARK SCHEME ===

**Answer.**

You get full points if you included some interpretation like this:

> There are relatively fewer accidents on weekends than weekdays.

=== END MARK SCHEME ===

**Your answer here**.

### Exercise 7

#### 7.1

Calculate the total number of accidents for each borough.

In [ ]:
def ex_7_1(df):
    """
    Group accidents by borough
    
    Arguments:
    `df`: A pandas DataFrame
    
    Outputs:
    `boroughs`: The grouped Series
    """
    
    ### BEGIN SOLUTION
    
    boroughs = df.groupby('BOROUGH').size()
    
    ### END SOLUTION
    
    return boroughs

#### 7.2
##### 7.2.1

Plot a bar graph of the previous data.

In [ ]:
ex_7_1(df).plot.bar()

##### 7.2.2

What do you notice in the plot?

=== BEGIN MARK SCHEME ===

**Answer.**

You get full points if you included some interpretation like this:

> We can see that Brooklyn and Queens have a very high number of accidents relative to the other three boroughs.

=== END MARK SCHEME ===

**Your answer here**.

#### 7.3 (hard)

How about per square mile? Calculate the number accidents per square mile for each borough.

**Hint:** You will have to update the keys in the borough dictionary to match the names in the DataFrame.

In [ ]:
def ex_7_3(df, borough_data):
    """
    Calculate accidents per sq mile for each borough
    
    Arguments:
    `borough_frame`: A pandas DataFrame with the count of accidents per borough
    `borough_data`: A python dictionary with population and area data for each borough
    
    
    Outputs:
    `borough_frame`: The same `borough_frame` DataFrame used as input, only with an
    additional column called `accidents_per_sq_mi` that results from dividing
    the number of accidents in each borough by its area. Please call this new column
    exactly `accidents_per_sq_mi` - otherwise the test cells will throw an error.
    """
    
    boroughs = ex_7_1(df)
    borough_frame = pd.DataFrame(boroughs)
    
    ### BEGIN SOLUTION
    # Since there are differences in the text used in the data and Wikipedia data, let's update it
    borough_data['bronx'] = borough_data.pop('the bronx')    
    
    # Making the keys in borough_frame lowercase
    borough_frame.columns = ['count']
    borough_frame['borough'] = borough_frame.index.str.lower()
    
    # Calculating accidents per square mile
    borough_frame['accidents_per_sq_mi'] = borough_frame.apply(lambda x: x['count'] / borough_data[x['borough']]['area'], axis=1)
    ### END SOLUTION
    
    return borough_frame # This must be a DataFrame, NOT a Series

#### 7.4
##### 7.4.1

Plot a bar graph of the accidents per square mile per borough with the data you just calculated. 

In [ ]:
ex_7_3(df, borough_data).plot.bar(x='borough', y='accidents_per_sq_mi', legend=False, title="Accidents per sq mi")

##### 7.4.2

What can you conclude?

=== BEGIN MARK SCHEME ===

**Answer.**

You get full points if you included some interpretation like this:

> When looking at the `accidents_per_sq_mi` parameter, Manhattan tops the list by a wide margin. This clearly shows that even though Brooklyn and Queens have more total accidents, Manhattan has a much higher concentration of accidents.

=== END MARK SCHEME ===

**Your answer here**.

### Exercise 8

#### 8.1

Create a Series of the number of accidents per hour and borough.

In [ ]:
def ex_8_1(df):
    """
    Calculate accidents per hour for each borough
    
    Arguments:
    `df`: A pandas DataFrame
    
    
    Outputs:
    `bor_hour`: A Series. This should be the result of doing groupby by borough
    and hour.
    """
    
    ### BEGIN SOLUTION
    df['TIME']=pd.to_datetime(df['TIME'])
    df['HOUR'] = df['TIME'].dt.hour
    bor_hour = df.groupby(['BOROUGH', 'HOUR']).size()    
    ### END SOLUTION
    
    return bor_hour

#### 8.2
##### 8.2.1

Plot a bar graph for each borough showing the number of accidents for each hour of the day. 

In [ ]:
bor_hour = ex_8_1(df).to_frame().reset_index()
bor_hour = bor_hour.rename(columns={0:"count"})
chart = sns.FacetGrid(bor_hour, col='BOROUGH', margin_titles=True, col_wrap=3, aspect=2, row_order=df['BOROUGH'].unique)
chart.map(sns.barplot, 'HOUR', 'count',)

##### 8.2.2

Which hours have the most accidents for each borough?

**Hint:** You can use ```sns.FacetGrid``` to create a grid of plots with the hourly data of each borough.

=== BEGIN MARK SCHEME ===

**Answer.**

You get full points if you included some interpretation like this:

> We can see that in all the boroughs the accident count is highest from approximately 2 - 6PM. But in Manhattan and the Bronx, you can see that there is not as much of a relative increase during these hours as in Brooklyn or Queens. Additionally, Staten Island has the lowest overall number of accidents.

=== END MARK SCHEME ===

**Your answer here**.

### Exercise 9 (hard)

Using `contrib_df`, find which 6 factors cause the most accidents. It is important that you avoid double counting the contributing factors of a single accident.

**Hint:** You can use the [**`pd.melt()`**](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) function to take a subset of `df` and convert it from [wide format to narrow format](https://en.wikipedia.org/wiki/Wide_and_narrow_data).

In [ ]:
def ex_9(df):
    """
    Finds which 6 factors cause the most accidents, without 
    double counting the contributing factors of a single accident.
    
    Arguments:
    `contrib_df`: A pandas DataFrame.
    
    Outputs:
    `factors_most_acc`: A pandas DataFrame. It has only 6 elements, which are,
    sorted in descending order, the contributing factors with the most accidents.
    The column with the actual numbers is named `index`.    
    """
    
    ### BEGIN SOLUTION
    contrib_fact_cols = ['CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2','CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4','CONTRIBUTING FACTOR VEHICLE 5']
    contrib_df = pd.melt(df.reset_index(), id_vars="index", value_vars=contrib_fact_cols).dropna()
    contrib_df.head(10)
    
    # We drop the "variable" column - we don't need it here
    factors_most_acc = contrib_df.drop(columns="variable").copy()

    # This is the crucial step - removing duplicates so that we don't count the same accident:factor pair more than once
    factors_most_acc = factors_most_acc.drop_duplicates()

    # Good old groupby, etc.
    factors_most_acc = factors_most_acc.groupby("value").count()
    factors_most_acc = factors_most_acc.sort_values(by="index", ascending=False).head(6)
    
    ### END SOLUTION
    
    return factors_most_acc

### Exercise 10 (hard)

Which 10 vehicle type-borough pairs are most involved in accidents? Avoid double counting the types of vehicles involved in a single accident. You can apply a similar approach to the one used in the previous exercise using `pd.melt()`.

**Hint:** You may want to include `BOROUGH` as one of your `id_vars` (the other being `index`) in `pd.melt()`. Including `BOROUGH` in your final `.groupby()` is also a good idea.

In [ ]:
def ex_10(df):
    """
    Finds the 10 borough:vehicle type pairs with more accidents, without 
    double counting the vehicle types of a single accident.
    
    Arguments:
    `df`: A pandas DataFrame.
    
    Outputs:
    `vehi_most_acc`: A pandas DataFrame. It has only 10 elements, which are,
    sorted in descending order, the borough-vehicle pairs with the most accidents.
    The column with the actual numbers is named `index`
    """

    vehi_cols = ['VEHICLE TYPE CODE 1','VEHICLE TYPE CODE 2','VEHICLE TYPE CODE 3','VEHICLE TYPE CODE 4','VEHICLE TYPE CODE 5']
    
    ### BEGIN SOLUTION
    
    vehi_df = pd.melt(df.reset_index(), id_vars=["index","BOROUGH"], value_vars=vehi_cols).dropna()

    # We drop the "variable" column - we don't need it here
    vehi_most_acc = vehi_df.drop(columns="variable").copy()

    # This is the crucial step - removing duplicates so that we don't count
    # the same accident:borough:vehicle_type triplet more than once
    vehi_most_acc = vehi_most_acc.drop_duplicates()
    vehi_most_acc

    # Good old groupby, etc.
    vehi_most_acc = vehi_most_acc.groupby(["BOROUGH","value"]).count()
    vehi_most_acc = vehi_most_acc.sort_values(by="index", ascending=False).head(10)
    
    ### END SOLUTION
    
    return vehi_most_acc

### Exercise 11

In a 2018 [interview](https://www.nytimes.com/2019/01/01/nyregion/traffic-deaths-decrease-nyc.html) with The New York Times, New York's mayor de Blasio stated that "*Vision Zero is clearly working*". That year, the number of deaths in traffic accidents in NYC dropped to a historically low 202. Yet, as reported by [am New York Metro](https://www.amny.com/news/vision-zero-de-blasio-1-30707464/), the number of fatalities has increased by 30% in the first quarter of 2019 compared to the previous year and the number of pedestrians and cyclists injured has not seen any improvement.

Which of the following BEST describes how you would use the provided data to understand what went wrong in the first quarter of 2019? Please explain the reasons for your choice.

<ul>
A. Consider the accidents of the first quarter of 2019. Then, check for the most common causes of accidents where pedestrians and cyclists were involved. Give a recommendation based solely on this information.<br>
B. Create a pair of heat maps of the accidents involving injured/killed pedestrians and cyclists in the first quarter of 2018 and 2019. Compare these two to see if there is any change in the concentration of accidents. In critical areas, study the type of factors involved in the accidents. Give a recommendation to visit these areas to study the problem further.<br>
C. The provided data is insufficient to improve our understanding of the situation.<br>
D. None of the above. (If you choose this, please elaborate on what you would do instead.)<br>
</ul>


=== BEGIN MARK SCHEME ===

**Answer.** You get half the points if you chose B, and the other half for giving a convincing argument for it. If you chose D, you will get full points if you give a convincing argument. Here are some thoughts:

* A is not viable as we are not taking into account possible changes the Vision Zero program implemented after the first quarter of 2018.

* B is more nuanced as it is trying to understand first what changes might had happened after the first quarter of 2018. Then, it concentrates in important areas of the city and then suggests a visit to improve understanding of the situation. 

* C is not correct. Even though the provided data may not completely answer what went wrong in the first quarter of 2019, it may provide hints of where to look and avoid unnecessary work.

* D may or may not be valid, depending on what you wrote. There is not always a single approach for diagnosing problems like this one.

=== END MARK SCHEME ===

**Your answer here**.

### Exercise 12 (hard)

#### 12.1

Calculate the number of deaths caused by each type of vehicle.

**Hint 1:** As an example of how to compute vehicle involvement in deaths, suppose two people died in an accident where 5 vehicles were involved, and 4 are PASSENGER VEHICLE and 1 is a SPORT UTILITY/STATION WAGON. Then we would add two deaths to both the PASSENGER VEHICLE and SPORT UTILITY/STATION WAGON types.)

**Hint 2:** You will need to use `pd.melt()` and proceed as in the previous exercises to avoid double-counting the types of vehicles (i.e. you should remove duplicate "accident ID - vehicle type" pairs).

In [ ]:
def ex_12(df):
    """
    Calculate total killed per vehicle type and plot the result
    as a bar graph
    
    Arguments:
    `df`: A pandas DataFrame.    
    
    Outputs:
    `result`: A pandas DataFrame. Its index should be the vehicle type. Its only
    column should be `TOTAL KILLED`
    """
    
    ### BEGIN SOLUTION
    
    # This code computes the TOTAL KILLED column and removes accidents with no deaths 
    df["TOTAL KILLED"] = df[['NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST KILLED']].apply(sum, axis=1)
    rows_with_deaths = df["TOTAL KILLED"]>0
    df = df[rows_with_deaths]
    
    # Keeping only relevant columns
    keep_columns = ['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3',
           'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5','TOTAL KILLED']
    df = df[keep_columns].reset_index()
    
    # Melting and cleaning
    df = pd.melt(df, id_vars=["index", "TOTAL KILLED"], value_vars=keep_columns.remove("TOTAL KILLED"))
    df = df.drop(columns="variable")
    df = df.dropna(subset=["value"])
    df = df.drop_duplicates()
    df = df.groupby("value")[["TOTAL KILLED"]].sum()
    result = df.sort_values(by="TOTAL KILLED", ascending=False)
    ### END SOLUTION
    
    return result

#### 12.2
##### 12.2.1

Plot a bar chart for the top 5 vehicles.

In [ ]:
ex_12(df).plot.bar()

##### 12.2.2

Which vehicles are most often involved in deaths, and by how much more than the others?

=== BEGIN MARK SCHEME ===

**Answer.**

You get full points if you included some interpretation like this:

> It appears that `Station Wagon/Sport Utility Vehicle` and `Sedan` cause the most deaths. The former causes 4 times as many deaths as the other vehicles, and the latter causes 3 times as many deaths as the other vehicles. ```SPORT UTILITY / STATION WAGON``` is essentially the same as ```Station Wagon/Sport Utility Vehicle```, so these should be considered as one.

=== END MARK SCHEME ===

**Your answer here**.

## Testing cells

In [ ]:
# Ex. 2
assert type(ex_2(df)) == type(pd.Series([9,1,2])), "Ex. 2 - Your output isn't a pandas Series. If you use .groupby(), it outputs a Series by default."
assert ex_2(df).loc["2018-10"] == 13336, "Ex. 2 - Wrong output! Try using the .size() aggregation function with your .groupby()."
print("Exercise 2.1 looks correct!")

In [ ]:
# Ex 4
assert type(ex_4(df)) == type(pd.Series([9,1,2])), "Ex. 4 - Your output isn't a pandas Series. If you use .groupby(), it outputs a Series by default."
assert ex_4(df).loc[13] == 14224, "Ex. 4 - Wrong output! Try using the .size() aggregation function with your .groupby()."
print("Exercise 4.1 looks correct!")

In [ ]:
# Ex. 6
assert type(ex_6(df)) == type(pd.Series([9,1,2])), "Ex. 6 - Your output isn't a pandas Series. If you use .groupby(), it outputs a Series by default."
assert max(ex_6(df)) == 37886, "Ex. 6 - Your results don't match ours! Remember that you can use the .size() aggregation function to count the number of elements in a groupby group."
print("Exercise 6.1 looks correct!")

In [ ]:
# Ex. 7.1
assert type(ex_7_1(df)) == type(pd.Series([9,1,2])), "Ex. 7.1 - Your output isn't a pandas Series. If you use .groupby(), it outputs a Series by default."
assert max(ex_7_1(df)) == 76253, "Ex. 7.1 - Your results don't match ours! Remember that you can use the .size() aggregation function to count the number of elements in a groupby group."
print("Exercise 7.1 looks correct!")

In [ ]:
# Ex. 7.3
with open('data/borough_data.json') as f:
    borough_data=json.load(f)
borough_data
e73 = ex_7_3(df, borough_data)
assert "accidents_per_sq_mi" in e73.columns, "Ex. 7.3 - You didn't create an 'accidents_per_sq_mi' in your DataFrame!"
assert round(min(e73["accidents_per_sq_mi"])) == 149, "Ex. 7.3 - Your output doesn't match ours! Remember that you need to divide the number of accidents in each of the five boroughs by the respective areas in square miles."
print("Exercise 7.3 looks correct!")

In [ ]:
# Ex. 8.1
assert type(ex_8_1(df)) == type(pd.Series([9,1,2])), "Ex. 9 - Your output isn't a pandas Series. If you use .groupby(), it outputs a Series by default."
assert ex_8_1(df).max() == 5701, "Ex. 8.1 - Your numbers don't match ours. If you haven't already, you can try using .size() as your aggregation function."
print("Exercise 8.1 looks correct!")

In [ ]:
# Ex. 9
assert type(ex_9(df)) == type(pd.Series([9,1,2]).to_frame()), "Ex. 9 - Your output isn't a pandas DataFrame. If you use .groupby(), it outputs a Series by default."
assert len(ex_9(df)) == 6, "Ex. 9 - Your output doesn't have six elements. Did you forget to use .head(6)?"
assert int(ex_9(df).sum()) == 316248, "Ex. 9 - Your numbers don't match ours. Are you sure you sorted your Series in descending order? If you haven't already, you can try using .count() as your aggregation function."
print("Exercise 9 looks correct!")

In [ ]:
# Ex. 10
assert type(ex_10(df)) == type(pd.Series([9,1,2]).to_frame()), "Ex. 10 - Your output isn't a pandas DataFrame. If you use .groupby(), it outputs a Series by default."
assert len(ex_10(df)["index"]) == 10, "Ex. 10 - Your output doesn't have 10 elements. Did you forget to use .head(10)?"
assert ex_10(df)["index"].sum() == 229882, "Ex. 10 - Your numbers don't match ours. Are you sure you sorted your Series in descending order? If you haven't already, you can try using .count() as your aggregation function."
print("Exercise 10 looks correct!")

In [ ]:
# Ex. 12
e12 = ex_12(df)
assert type(e12) == type(pd.Series([9,1,2]).to_frame()), "Ex. 12 - Your output isn't a pandas DataFrame. If you use .groupby(), it outputs a Series by default."
assert int(e12.loc["Bike"]) == 19, "Ex. 12 - Your output doesn't match ours! Remember that you need to remove the duplicate pairs and use the .sum() aggregation function in your groupby."
print("Exercise 12.1 looks correct!")

## Attribution

"Vehicle Collisions in NYC 2015-Present", New York Police Department, [NYC Open Data terms of use](https://opendata.cityofnewyork.us/overview/#termsofuse), https://www.kaggle.com/nypd/vehicle-collisions

"Boroughs of New York City", Creative Commons Attribution-ShareAlike License, https://en.wikipedia.org/wiki/Boroughs_of_New_York_City